In [17]:
import numpy as np
import cv2
import pandas as pd
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [27]:
#Read in file path for spreadsheet with lesion counts per body site for a patient
#Make sure it is a .xlsx
file_path = r'M:\\PALM\\REDCapReports\\DailyTargetedLesionAssessment\\DailyTargetedLesionAssessment_240329_SilaphetTyra.xlsx'
data = pd.read_excel(file_path, sheet_name='DailyTargetedLesionAssessment_2')

# Specify the patient ID and the pattern for the event name
patient_id = "01-0004"
event_name_pattern = ".*2.*"  # Replace the X with the follow-up visit number
# For example, follow-up visit Day 12 will be: ".*12.*"

# Select only the row with the specified patient ID and event name pattern
selected_row = data[(data['PID or Screening ID:'] == patient_id) & (data['Event Name'].str.contains(event_name_pattern))].head(1)

# Set the index name explicitly to match the patient ID
selected_row = selected_row.rename_axis(index={selected_row.index[0]: patient_id})
print(selected_row)

   PID or Screening ID:            Event Name  Repeat Instrument  \
46              01-0004  DAY 2 (Arm 1: Study)                NaN   

    Repeat Instance  \
46              NaN   

      1. Une évaluation des lésions ciblées a-t-elle été effectuée? Si oui, indiquez le nombre de lésions pour les régions ci-des-sous.  Was a targeted lesion assessment performed? If yes, provide lesion counts for the regions below.     \
46                                 Oui (<i> Yes </I>)                                                                                                                                                                                          

    1.1 Anterior arm (assessment region) Anterior arm (assessment region) ND  \
46                                  42.0                                 NaN   

    1.2 Posterior Arm posterior arm ND  1.3 Axilla  ... Shin & Kneee ND  \
46               53.0              NaN         4.0  ...             NaN   

    1.8 Posterior Thigh

In [28]:
# Check if selected_row is empty
if selected_row.empty:
    print("No value")
else:
    # Convert columns to integers after handling non-finite values
    selected_row['1.1 Anterior arm (assessment region)'] = pd.to_numeric(selected_row['1.1 Anterior arm (assessment region)'], errors='coerce').fillna(0).astype(int)
    selected_row['1.3 Axilla'] = pd.to_numeric(selected_row['1.3 Axilla'], errors='coerce').fillna(0).astype(int)

    # Combined R Axilla with R Anterior Arm
    selected_row['CombinedR'] = selected_row['1.1 Anterior arm (assessment region)'] + selected_row['1.3 Axilla']

    # Drop the unnecessary columns
    lesion_data_cleancolumns = selected_row.drop(columns=['Event Name', 'Repeat Instrument', 'Repeat Instance',  
                                                     'Anterior arm (assessment region) ND', 'posterior arm ND', 'Axilla ND', 'Palm ND', 'Dorsal Hand ND', 'Anterior Thigh ND', 'Shin & Kneee ND', 'Posterior Thigh ND', 'Calf & Back of knee ND',
                                                     'Dorsal Foot ND', 'Sole ND', 'Complete?', '1.1 Anterior arm (assessment region)', '1.3 Axilla',
                                                     '   1. Une évaluation des lésions ciblées a-t-elle été effectuée? Si oui, indiquez le nombre de lésions pour les régions ci-des-sous.  Was a targeted lesion assessment performed? If yes, provide lesion counts for the regions below.   '])

    # Rename Columns to match M Drive 
    lesion_data_cleancolumns.rename(columns={'1.2 Posterior Arm':'RightPosteriorArm', 
                                           '1.4 palm':'RightPalm', 
                                           'Dorsal Hand':'RightDorsalHand', 
                                           '1.6 Anterior Thigh':'RightAnteriorThigh', 
                                           '1.7 Shin & Knee':'RightShin&Knee', 
                                           '1.8 Posterior Thigh':'RightPosteriorThigh',
                                           '1.9 Calf & back of knee':'RightCalf&BackofKnee',
                                           '1.10 Dorsal Foot':'RightDorsalFoot', 
                                           '1.11 Sole':'RightSole', 
                                           'CombinedR':'RightAnteriorArm'},
                                      inplace=True)

    # Convert columns to rows
    melted_df = lesion_data_cleancolumns.melt(id_vars=['PID or Screening ID:'], var_name='Body Site', value_name='Lesion Count')
    print(melted_df)
    
#If you see "NaN" as the Lesion Count, then it has no value. 

  PID or Screening ID:             Body Site  Lesion Count
0              01-0004     RightPosteriorArm          53.0
1              01-0004             RightPalm           2.0
2              01-0004       RightDorsalHand          15.0
3              01-0004    RightAnteriorThigh          41.0
4              01-0004        RightShin&Knee          32.0
5              01-0004   RightPosteriorThigh          46.0
6              01-0004  RightCalf&BackofKnee          24.0
7              01-0004       RightDorsalFoot           2.0
8              01-0004             RightSole           0.0
9              01-0004      RightAnteriorArm          46.0


In [22]:
from PIL import Image, ImageOps, ImageDraw
import pandas as pd
import os

data = melted_df

# Specify the folder paths for input and output images
input_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\FilledMasks\\AllSites\\'
output_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\'
overlay_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\'

# Color scale definition
# Have to make 0, 0.1 off white
color_scale = {
    (0, 0.1): "rgb(253, 253, 253)",
    (1, 50): "rgb(245, 171, 178)",
    (51, 100): "rgb(255, 2, 0)",
    (101, 150): "rgb(209, 1, 0)",
    (151, 200): "rgb(140, 0, 0)",
    (201, 250): "rgb(71, 0, 1)",
    (251, 500): "rgb(0, 0, 0)" 
}

# Function to map lesion count to color based on the color scale
def map_lesion_count_to_color(lesion_count):
    for range_, color in color_scale.items():
        if range_[0] <= lesion_count <= range_[1]:
            return color
    return "black"

# Iterate through each row in the spreadsheet
for index, row in data.iterrows():
    # Extract body site and lesion count
    body_site = row['Body Site']
    lesion_count = row['Lesion Count']

    # Construct the file path for the corresponding binary mask
    #HAS TO BE WHITE BG
    binary_mask_path = f'{input_folder}{body_site}_WhiteBG.png'
    
    # Open the corresponding binary mask
    binary_mask = Image.open(binary_mask_path).convert("L")
    
    # Map lesion count to color
    color = map_lesion_count_to_color(lesion_count)

    # Create a copy of the binary mask and fill in black pixels with the color
    colored_mask = ImageOps.colorize(binary_mask, black=color, white="white")
    
    # Ensure the colored mask has RGBA mode
    colored_mask = colored_mask.convert("RGBA")

    # Remove the background by setting alpha channel to 0 for white pixels
    data = colored_mask.getdata()
    new_data = [(r, g, b, 0) if r == 255 and g == 255 and b == 255 else (r, g, b, a) for r, g, b, a in data]
    colored_mask.putdata(new_data)

    # Save the colored mask without background
    output_path = f'{output_folder}{body_site}_ColoredOutput.png'
    colored_mask.save(output_path, 'PNG')
    print(f"Processed: {body_site}")

print("Done")

Processed: RightPosteriorArm
Processed: RightPalm
Processed: RightDorsalHand
Processed: RightAnteriorThigh
Processed: RightShin&Knee
Processed: RightPosteriorThigh
Processed: RightCalf&BackofKnee
Processed: RightDorsalFoot
Processed: RightSole
Processed: RightAnteriorArm
Done


In [14]:
from PIL import Image

# List of image pairs: (first_image_path, second_image_path, output_path)
image_pairs = [
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightAnteriorArm_ColoredOutput.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPosteriorArm_ColoredOutput.png", 
     f'{overlay_folder}RightAnteriorArmPosteriorArm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightAnteriorArmPosteriorArm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPalm_ColoredOutput.png", 
     f'{overlay_folder}RightPalm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightPalm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightDorsalHand_ColoredOutput.png", 
     f'{overlay_folder}RightDorsalHand_Overlayed.png'), 
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightDorsalHand_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightAnteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}RightAnteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightAnteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightShin&Knee_ColoredOutput.png", 
     f'{overlay_folder}RightShin&Knee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightShin&Knee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPosteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}RightPosteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightPosteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightCalf&BackofKnee_ColoredOutput.png", 
     f'{overlay_folder}RightCalf&BackofKnee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightCalf&BackofKnee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightDorsalFoot_ColoredOutput.png", 
     f'{overlay_folder}RightDorsalFoot_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightDorsalFoot_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightSole_ColoredOutput.png", 
     f'{overlay_folder}RightSole_Overlayed.png'),
    
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\UnfilledMasks\\Blank\\AllSites_AllWhite.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightSole_Overlayed.png", 
     f'{overlay_folder}WhiteBG_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\WhiteBG_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\UnfilledMasks\\Blank\\TargetedAvatar_NoBG.png", 
     f'{overlay_folder}{patient_id}_Targeted.png'), 
    
]

# Iterate through each image pair
for first_image_path, second_image_path, output_path in image_pairs:
    # Loading the first image (with transparent BG)
    img1 = Image.open(first_image_path).convert("RGBA")

    # Loading the second image (with transparent BG)
    img2 = Image.open(second_image_path).convert("RGBA")
    img2 = img2.resize(img1.size)

    # Using alpha_composite to overlay the images onto each other
    img3 = Image.alpha_composite(img1, img2)
    img3.save(output_path, 'PNG')
    
img3.show()

print("Done")

Done
